<h2><center>Procedure for Estimation and Reporting of Uncertainty Due to Discretization
in CFD Applications</center></h2>

<h4><center>Aerodynamic evaluation of modified SG6043 airfoils using
the γ − Reθ turbulence model with OpenFOAM</center></h4>

This notebook implements the procedure for uncertainty quantification due to meshing in a CFD application according to the Fluids Engineering Division of ASME. The recommended procedure for the estimation of discretization error is as follows [1]:

Step 1: Define a representative mesh / cell / grid size $h$,
$$
    h = \left[ \frac{1}{N}\sum_{i=1}^{N} (\Delta V_i) \right]^{1/3}\quad\mathrm{3D}
$$

$$
    h = \left[ \frac{1}{N}\sum_{i=1}^{N} (\Delta A_i) \right]^{1/2}\quad\mathrm{2D}
$$


Step 2: Select 3 significantly different sets of grids and run simulations to determine the values of key variables important to the objective of the simulation study. It is prefereble that the grid refinement factor satisfies the following condition:

$$
r = h_{\mathrm{coarse}}/h_{\mathrm{fine}} > 1.3
$$

Step 3: Let $h_1 < h_2 < h_3$, $r_{21} = h_2/h_1$, $r_{32} = h_3/h_2$, and $\varepsilon_{32}=\phi_3-\phi_2$, $\varepsilon_{21}=\phi_2-\phi_1$ calculate the apparent order p of the method using, 

$$
p = \frac{1}{\ln{(r_{21})}}|\ln|\varepsilon_{32}/\varepsilon_{21}|+q(p)|
$$

with, 

$$
q(p) = \ln\left(\frac{r_{21}^p-s}{r_{32}^p-s}\right)
$$

and 

$$
s = 1·\mathrm{sgn}(\varepsilon_{32}/\varepsilon_{21})
$$




Step 4: Calculate the extrapolated values from, 
$$
\phi_{\mathrm{ext}}^{21} = \frac{(r_{21}^p\phi_1-\phi_2)}{(r_{21}^p-1)} 
$$


Step 5: Calculate and report the following error estimates with the apparent order $p$, 

$$
e_a^{21} = \left| \frac{\phi_1-\phi_2}{\phi_1}\right|
$$

$$
e_{\mathrm{ext}}^{21} = \left| \frac{\phi_{\mathrm{ext}}^{21}-\phi_1}{\phi_{\mathrm{ext}}^{21}}\right|
$$

$$
GCI_{\mathrm{fine}}^{21} = \frac{1.25e_{a}^{21}}{r_{21}^p-1}
$$

In [2]:
# Array handling and numerical methods
import numpy as np
from scipy.optimize import fixed_point

# plotting and visualizing
import matplotlib.pyplot as plt

# OpenFOAM file handling
from PyFoam.RunDictionary.ParsedParameterFile import ParsedParameterFile

In [3]:
# Get values from OpenFOAM case

case_directory = "../runs"
meshcase1 = "/GRTsteady0Mesh0"
meshcase2 = "/GRTsteady0Mesh1" 
meshcase3 = "/GRTsteady0Mesh2" 

VolumeFile1 = ParsedParameterFile(case_directory+meshcase1+"/4000/V")
VolumeFile2 = ParsedParameterFile(case_directory+meshcase2+"/4000/V")
VolumeFile3 = ParsedParameterFile(case_directory+meshcase3+"/4000/V")

coeffFile1 = np.loadtxt(case_directory+meshcase1+
                        "/postProcessing/forceCoeffs/0/forceCoeffs.dat")
coeffFile2 = np.loadtxt(case_directory+meshcase2+
                        "/postProcessing/forceCoeffs/0/forceCoeffs.dat")
coeffFile3 = np.loadtxt(case_directory+meshcase3+
                        "/postProcessing/forceCoeffs/0/forceCoeffs.dat")

In [4]:
volumes1 = np.array(VolumeFile1["internalField"])
volumes2 = np.array(VolumeFile2["internalField"])
volumes3 = np.array(VolumeFile3["internalField"])

h3 = np.mean(volumes1)**(1/3)
h2 = np.mean(volumes2)**(1/3)
h1 = np.mean(volumes3)**(1/3)

Cp3 = np.mean(coeffFile1, axis=0)[3]
Cp2 = np.mean(coeffFile2, axis=0)[3]
Cp1 = np.mean(coeffFile3, axis=0)[3]

In [5]:
print(h1,h2,h3)

0.011036442541895581 0.014584054838204618 0.018361206061764508


In [6]:
eps = [Cp2-Cp1, Cp3-Cp2]
r = [h2/h1, h3/h2]
print(r)

[1.3214452739496354, 1.2589918418069306]


In [7]:
def q(p, r, s):
    return np.log((r[0]**p-s)/(r[1]**p-s))
                                   
def p(p, eps, r):
    feps = eps[1]/eps[0]
    resp = (1/np.log(r[0]))*np.abs(np.log(np.abs(feps))+q(p, r, np.sign(feps)))
                                      
    return resp

In [8]:
# p - order solution by minimization

feps = eps[1]/eps[0]
p0 = (1/np.log(r[0]))*np.abs(np.log(np.abs(feps)))
px = fixed_point(p, p0, (eps, r))
print(px)

3.001893598175936


In [10]:
Cpext21 = (r[0]**px*Cp1-Cp2)/(r[0]**px-1) 
print(Cpext21)

0.42694368746092365


[1]“Procedure for Estimation and Reporting of Uncertainty Due to Discretization in CFD Applications” 
Journal of Fluids Engineering, vol. 130, no. 7. ASME International, p. 078001, 2008. doi: 10.1115/1.2960953.